# 17. Modelos de regresion con LSTM
***


### Caso de estudio #1



### Tecnicas de ajuste de los datos y modelos:

| Ingeniería de datos       | Si / No       |
| :-------                  | :------:    |
| Escalado de datos         | Si          |
| Sobremuestreo  | No        |
| Incluir variable ciclo | Si        |


In [1]:
# librerias
import numpy as np
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns

from numpy.random import seed
seed(0)

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, recall_score, precision_score

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score
from plot_metric.functions import BinaryClassification
from sklearn.metrics import classification_report

keras.backend.clear_session()
import gc
gc.collect()

import os
import h5py
from sklearn import datasets 
from keras.models import model_from_json

from numba import cuda
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from keras.models import load_model
from sklearn.metrics import mean_absolute_error
from keras.layers import Bidirectional
from keras.layers import TimeDistributed


Using TensorFlow backend.


In [40]:
# funcion para calcular metricas de desempeño

def regression_metrics(y_test, y_pred2):
    # Mean Deviation (MD)
    MD=np.mean(y_test - y_pred2)
    # mean absolute error
    MAE=mean_absolute_error(y_test,y_pred2)
    # Mean Squared Error (MSE)
    MSE=mean_squared_error(y_test,y_pred2)
    # Root Mean Squared Error (RMSE)
    RMSE= sqrt(mean_squared_error(y_test,y_pred2))
    # Mean Percent Error (MPE)
    MPE=np.mean((y_test - y_pred2) / y_test) * 100
    # Mean Absolute Percent Error (MAPE)
    MAPE=np.mean(np.abs((y_test - y_pred2) / y_test)) * 100
    print('\n')
    print("METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION")
    print('-'*60)
    print(f'Mean Deviation (MD): {round(MD,2)}') 
    print(f'Mean Absolute Error (MAE): {round(MAE,2)}')
    print('-'*60)
    print(f'Mean Squared Error (MSE): {round(MSE,2)}') 
    print(f'Root Mean Squared Error (RMSE): {round(RMSE,2)}')
    print('-'*60)    
    print(f'Mean Percent Error (MPE): {round(MPE,2)}') 
    print(f'Mean Absolute Percent Error (MAPE): {round(MAPE,2)}')
    print('-'*60)


### Preparar datos

In [2]:
columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
train.head()


,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [3]:
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
test.head()


,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,138


In [4]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [5]:
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']

objetivo='ttf'

In [6]:
# normalizado de datos 
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])

In [7]:
train.head()

,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,0.529086
1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,0.526316
2,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,0.523546
3,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,0.520776
4,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,0.518006


In [8]:
test.head()

,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,0.00000,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,1.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.393352
1,0.00277,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,1.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.390582
2,0.00554,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,1.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.387812
3,0.00831,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,1.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.385042
4,0.01108,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,1.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.382271


In [9]:
print("numero de dimensiones =", train.ndim)
print("tamaño de la tabla =", train.shape)
print("total datos =", train.size)

numero de dimensiones = 2
tamaño de la tabla = (20631, 26)
total datos = 536406


In [10]:
print("numero de dimensiones =", test.ndim)
print("tamaño de la tabla =", test.shape)
print("total datos =", test.size)

numero de dimensiones = 2
tamaño de la tabla = (13096, 26)
total datos = 340496


In [11]:
x_train=train.iloc[:,np.r_[0:25]]
x_train.head()


,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662
1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014
2,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375
3,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386
4,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502


In [12]:
x_test=test.iloc[:,np.r_[0:25]]
x_test.head()

,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,0.00000,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,1.0,...,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834
1,0.00277,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,1.0,...,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827
2,0.00554,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,1.0,...,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348
3,0.00831,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,1.0,...,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110
4,0.01108,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,1.0,...,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377


In [13]:
y_train=train.iloc[:,25]
y_train.head()

0    0.529086
1    0.526316
2    0.523546
3    0.520776
4    0.518006
Name: ttf, dtype: float64

In [14]:
y_test=test.iloc[:,25]
y_test.head()

0    0.393352
1    0.390582
2    0.387812
3    0.385042
4    0.382271
Name: ttf, dtype: float64

## Preparar datos para LSTM (Reshape Subsequences)

In [15]:
# convert to numpy array
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
type(x_test)

numpy.ndarray

In [16]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(20631, 25)
(20631,)
(13096, 25)
(13096,)


In [17]:
# tecnica para convertir datos 2d a 3d (observaciones, variables) => (observaciones, pasos de tiempo, variables)
# reshape  = [samples, timesteps, features] 
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


In [18]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(20631, 1, 25)
(20631,)
(13096, 1, 25)
(13096,)


In [19]:
print("numero de dimensiones x_train =", x_train.ndim)
print("tamaño de la tabla x_train =", x_train.shape)
print("total datos x_train =", x_train.size)
print('-'*60)
print("numero de dimensiones y_train =", y_train.ndim)
print("tamaño de la tabla y_train =", y_train.shape)
print("total datos y_train =", y_train.size)
print('-'*60)
print("numero de dimensiones x_test =", x_test.ndim)
print("tamaño de la tabla x_test =", x_test.shape)
print("total datos x_test =", x_test.size)
print('-'*60)
print("numero de dimensiones y_test =", y_test.ndim)
print("tamaño de la tabla y_test =", y_test.shape)
print("total datos y_test =", y_test.size)


numero de dimensiones x_train = 3
tamaño de la tabla x_train = (20631, 1, 25)
total datos x_train = 515775
------------------------------------------------------------
numero de dimensiones y_train = 1
tamaño de la tabla y_train = (20631,)
total datos y_train = 20631
------------------------------------------------------------
numero de dimensiones x_test = 3
tamaño de la tabla x_test = (13096, 1, 25)
total datos x_test = 327400
------------------------------------------------------------
numero de dimensiones y_test = 1
tamaño de la tabla y_test = (13096,)
total datos y_test = 13096


# Modelos de regresion 

# Vanilla LSTM

In [21]:
# diseñar la red
model_path_clf = 'regresion2_model.h5'  
model = Sequential()
model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# entrenar el modelo
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())
 





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20631 samples, validate on 13096 samples
Epoch 1/100
 - 9s - loss: 0.1081 - val_loss: 0.1511

Epoch 00001: val_loss improved from inf to 0.15108, saving model to regresion2_model.h5
Epoch 2/100
 - 4s - loss: 0.0925 - val_loss: 0.1157

Epoch 00002: val_loss improved from 0.15108 to 0.11572, saving model to regresion2_model.h5
Epoch 3/100
 - 4s - loss: 0.0910 - val_loss: 0.0993

Epoch 00003: val_loss improved from 0.11572 to 0.09930, saving model to regresion2_model.h5
Epoch 4/100
 - 4s - loss: 0.0903 - val_loss: 0.0954

Epoch 00004: val_loss improved from 0.09930 to 0.09543, saving model to regresion2_model.h5
Epoch 5/100
 - 4s - loss: 0.0895 - val_loss: 0.0948

Epoch 00005: val_loss improved from 0.09543 to 0.09482, saving model to regresion2_model.h5
Epoch 6/100
 - 5s - loss: 0.0887 - val_loss: 0.0952

Epoch 00006: val_loss did not improve from 0.09482
Epoch 7/100
 - 4s - loss: 0.

In [21]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion2_model.h5')

In [22]:
# realizar prediccion 
yhat = saved_model.predict(x_test)


In [23]:
yhat.shape

(13096, 1)

In [24]:
yhat

array([[0.5685879 ],
       [0.54165906],
       [0.51711226],
       ...,
       [0.0749625 ],
       [0.06760633],
       [0.04658365]], dtype=float32)

Dado que los valores de la predicccion estan en formato 3d, se procede a convertir a 2d

In [25]:
# 3d a 2d
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [26]:
y_test = y_test.reshape((len(y_test), 1))

In [27]:
y_test.shape

(13096, 1)

In [28]:
# calculate RMSE en formato normalizado
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.115


Dado que los valores de prediccion estan normalizados, se procede a invertir la escala para convertirlos en formato ciclos

In [29]:
# invertir escala del pronostico

inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [30]:
inv_yhat.shape

(13096, 26)

In [31]:
inv_yhat[0]

array([0.5685879 , 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [32]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [33]:
inv_yhat2

array([[ 2.06260232e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.96538920e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.87677524e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 2.80614613e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 2.54058851e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 1.78166986e+01,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [34]:
inv_yhat3 = inv_yhat2[:,0]

In [35]:
# y_test pronosticado transformado
inv_yhat3

array([206.26023161, 196.53891963, 187.67752409, ...,  28.06146133,
        25.4058851 ,  17.81669855])

In [36]:
# y_test transformado

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [41]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 8.48
Mean Absolute Error (MAE): 30.57
------------------------------------------------------------
Mean Squared Error (MSE): 1718.37
Root Mean Squared Error (RMSE): 41.45
------------------------------------------------------------
Mean Percent Error (MPE): -2.88
Mean Absolute Percent Error (MAPE): 23.59
------------------------------------------------------------


### Preparar datos para el nuevo modelo

In [82]:
# Preparar datos de nuevo para el modelo, ya que se habian modificado por escalado y reshape
# no hay cambios en los datos, son las mismas lineas de la parte superior del codigo

columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']
objetivo='ttf'
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])
x_train=train.iloc[:,np.r_[0:25]]
x_test=test.iloc[:,np.r_[0:25]]
y_train=train.iloc[:,25]
y_test=test.iloc[:,25]
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


# Staked LSTM

In [62]:

model_path_clf = 'regresion3_model.h5'  
model = Sequential()
model.add(LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')



history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())

Train on 20631 samples, validate on 13096 samples
Epoch 1/50
 - 12s - loss: 0.1129 - val_loss: 0.1654

Epoch 00001: val_loss improved from inf to 0.16541, saving model to regresion3_model.h5
Epoch 2/50
 - 11s - loss: 0.0961 - val_loss: 0.1368

Epoch 00002: val_loss improved from 0.16541 to 0.13677, saving model to regresion3_model.h5
Epoch 3/50
 - 10s - loss: 0.0938 - val_loss: 0.1155

Epoch 00003: val_loss improved from 0.13677 to 0.11548, saving model to regresion3_model.h5
Epoch 4/50
 - 11s - loss: 0.0920 - val_loss: 0.1090

Epoch 00004: val_loss improved from 0.11548 to 0.10904, saving model to regresion3_model.h5
Epoch 5/50
 - 11s - loss: 0.0911 - val_loss: 0.1052

Epoch 00005: val_loss improved from 0.10904 to 0.10519, saving model to regresion3_model.h5
Epoch 6/50
 - 10s - loss: 0.0903 - val_loss: 0.1026

Epoch 00006: val_loss improved from 0.10519 to 0.10257, saving model to regresion3_model.h5
Epoch 7/50
 - 10s - loss: 0.0898 - val_loss: 0.1008

Epoch 00007: val_loss improved 

In [44]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion3_model.h5')

In [45]:
# prediccion
yhat = saved_model.predict(x_test)


In [46]:
yhat.shape

(13096, 1)

In [47]:
yhat

array([[0.50465786],
       [0.5088497 ],
       [0.49690408],
       ...,
       [0.05174018],
       [0.04941075],
       [0.0305727 ]], dtype=float32)

In [48]:
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [49]:
y_test = y_test.reshape((len(y_test), 1))

In [50]:
y_test.shape

(13096, 1)

In [51]:
# calcular RMSE
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.119


In [52]:
from numpy import concatenate
inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [53]:
inv_yhat.shape

(13096, 26)

In [54]:
inv_yhat[0]

array([0.50465786, 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [55]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [56]:
inv_yhat2

array([[ 1.83181489e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.84694735e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.80382371e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 1.96782066e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 1.88372791e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 1.20367438e+01,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [57]:
inv_yhat3 = inv_yhat2[:,0]

In [58]:
inv_yhat3

array([183.18148911, 184.69473463, 180.38237113, ...,  19.67820658,
        18.83727913,  12.0367438 ])

In [59]:

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [60]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 13.29
Mean Absolute Error (MAE): 30.84
------------------------------------------------------------
Mean Squared Error (MSE): 1832.46
Root Mean Squared Error (RMSE): 42.81
------------------------------------------------------------
Mean Percent Error (MPE): 2.01
Mean Absolute Percent Error (MAPE): 22.72
------------------------------------------------------------


### Preparar datos para el nuevo modelo

In [247]:
# Preparar datos de nuevo para el modelo, ya que se habian modificado por escalado y reshape
# no hay cambios en los datos, son las mismas lineas de la parte superior del codigo

columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']
objetivo='ttf'
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])
x_train=train.iloc[:,np.r_[0:25]]
x_test=test.iloc[:,np.r_[0:25]]
y_train=train.iloc[:,25]
y_test=test.iloc[:,25]
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


# Staked LSTM con 4 capas ocultas

In [248]:
# design network
model_path_clf = 'regresion6_model.h5'  
model = Sequential()
model.add(LSTM(400, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')


# fit network
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())

Train on 20631 samples, validate on 13096 samples
Epoch 1/100
 - 20s - loss: 0.1213 - val_loss: 0.1680

Epoch 00001: val_loss improved from inf to 0.16801, saving model to regresion6_model.h5
Epoch 2/100
 - 14s - loss: 0.1013 - val_loss: 0.1348

Epoch 00002: val_loss improved from 0.16801 to 0.13481, saving model to regresion6_model.h5
Epoch 3/100
 - 14s - loss: 0.0972 - val_loss: 0.1193

Epoch 00003: val_loss improved from 0.13481 to 0.11930, saving model to regresion6_model.h5
Epoch 4/100
 - 14s - loss: 0.0948 - val_loss: 0.1123

Epoch 00004: val_loss improved from 0.11930 to 0.11226, saving model to regresion6_model.h5
Epoch 5/100
 - 14s - loss: 0.0935 - val_loss: 0.1022

Epoch 00005: val_loss improved from 0.11226 to 0.10217, saving model to regresion6_model.h5
Epoch 6/100
 - 14s - loss: 0.0925 - val_loss: 0.1034

Epoch 00006: val_loss did not improve from 0.10217
Epoch 7/100
 - 14s - loss: 0.0914 - val_loss: 0.0985

Epoch 00007: val_loss improved from 0.10217 to 0.09851, saving mo

In [249]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion6_model.h5')

In [250]:
# prediccion
yhat = saved_model.predict(x_test)


In [251]:
yhat.shape

(13096, 1)

In [252]:
yhat

array([[0.52207667],
       [0.523903  ],
       [0.50842625],
       ...,
       [0.03814742],
       [0.04011813],
       [0.02726528]], dtype=float32)

In [253]:
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [254]:
y_test = y_test.reshape((len(y_test), 1))

In [255]:
y_test.shape

(13096, 1)

In [256]:
# calcular RMSE
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.114


In [257]:
inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [258]:
inv_yhat.shape

(13096, 26)

In [259]:
inv_yhat[0]

array([0.52207667, 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [260]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [261]:
inv_yhat2

array([[ 1.89469677e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.90128987e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.84541876e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 1.47712185e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 1.54826442e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 1.08427663e+01,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [262]:
inv_yhat3 = inv_yhat2[:,0]

In [263]:
inv_yhat3

array([189.46967655, 190.12898743, 184.5418759 , ...,  14.77121851,
        15.48264423,  10.84276626])

In [264]:

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [265]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 7.67
Mean Absolute Error (MAE): 30.46
------------------------------------------------------------
Mean Squared Error (MSE): 1690.81
Root Mean Squared Error (RMSE): 41.12
------------------------------------------------------------
Mean Percent Error (MPE): -1.65
Mean Absolute Percent Error (MAPE): 23.21
------------------------------------------------------------


### Preparar datos para el nuevo modelo

In [126]:
# Preparar datos de nuevo para el modelo, ya que se habian modificado por escalado y reshape
# no hay cambios en los datos, son las mismas lineas de la parte superior del codigo

columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']
objetivo='ttf'
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])
x_train=train.iloc[:,np.r_[0:25]]
x_test=test.iloc[:,np.r_[0:25]]
y_train=train.iloc[:,25]
y_test=test.iloc[:,25]
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


## Encoder-Decoder LSTM With Multivariate Input

In [124]:
# y_train de 2d a 3d
y_train3d = y_train.reshape((y_train.shape[0], n_steps, 1)) 

In [125]:
y_train3d.shape

(20631, 1, 1)

In [126]:
# y_test de 2d a 3d
y_test3d = y_test.reshape((y_test.shape[0], n_steps, 1))
y_test3d.shape

(13096, 1, 1)

In [127]:
# modelo

from keras.layers import RepeatVector
from keras.layers import TimeDistributed

n_outputs=1
model_path_clf = 'regresion4_model.h5'  
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(RepeatVector(n_outputs))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mse', optimizer='adam') # con perdida mse


# entrenar la red
history = model.fit(x_train, y_train3d, epochs=100, batch_size=32, validation_data=(x_test, y_test3d), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())



Train on 20631 samples, validate on 13096 samples
Epoch 1/100
 - 10s - loss: 0.0229 - val_loss: 0.0341

Epoch 00001: val_loss improved from inf to 0.03409, saving model to regresion4_model.h5
Epoch 2/100
 - 8s - loss: 0.0157 - val_loss: 0.0303

Epoch 00002: val_loss improved from 0.03409 to 0.03026, saving model to regresion4_model.h5
Epoch 3/100
 - 8s - loss: 0.0146 - val_loss: 0.0307

Epoch 00003: val_loss did not improve from 0.03026
Epoch 4/100
 - 8s - loss: 0.0141 - val_loss: 0.0312

Epoch 00004: val_loss did not improve from 0.03026
Epoch 5/100
 - 8s - loss: 0.0136 - val_loss: 0.0243

Epoch 00005: val_loss improved from 0.03026 to 0.02428, saving model to regresion4_model.h5
Epoch 6/100
 - 8s - loss: 0.0133 - val_loss: 0.0223

Epoch 00006: val_loss improved from 0.02428 to 0.02229, saving model to regresion4_model.h5
Epoch 7/100
 - 8s - loss: 0.0131 - val_loss: 0.0209

Epoch 00007: val_loss improved from 0.02229 to 0.02086, saving model to regresion4_model.h5
Epoch 8/100
 - 8s - 

In [127]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion4_model.h5')

In [128]:
# prediccion
yhat = saved_model.predict(x_test)


In [129]:
yhat.shape

(13096, 1, 1)

In [130]:
yhat

array([[[0.52438605]],

       [[0.5307803 ]],

       [[0.51905566]],

       ...,

       [[0.07463536]],

       [[0.0654662 ]],

       [[0.02439013]]], dtype=float32)

In [131]:
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [132]:
y_test = y_test.reshape((len(y_test), 1))

In [133]:
y_test.shape

(13096, 1)

In [134]:
# convertir el pronostico a 2d
yhat = yhat.reshape((len(yhat), 1))

In [135]:
# calcular RMSE
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.114


In [136]:
inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [137]:
inv_yhat.shape

(13096, 26)

In [138]:
inv_yhat[0]

array([0.52438605, 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [139]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [140]:
inv_yhat2

array([[ 1.90303363e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.92611694e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.88379095e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 2.79433638e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 2.46332965e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 9.80483738e+00,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [141]:
inv_yhat3 = inv_yhat2[:,0]

In [142]:
inv_yhat3

array([190.30336344, 192.61169386, 188.3790949 , ...,  27.94336376,
        24.63329652,   9.80483738])

In [143]:

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [147]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): -1.62
Mean Absolute Error (MAE): 32.14
------------------------------------------------------------
Mean Squared Error (MSE): 1688.19
Root Mean Squared Error (RMSE): 41.09
------------------------------------------------------------
Mean Percent Error (MPE): -9.71
Mean Absolute Percent Error (MAPE): 26.61
------------------------------------------------------------


### Preparar datos para el nuevo modelo

In [221]:
# Preparar datos de nuevo para el modelo, ya que se habian modificado por escalado y reshape
# no hay cambios en los datos, son las mismas lineas de la parte superior del codigo

columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']
objetivo='ttf'
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])
x_train=train.iloc[:,np.r_[0:25]]
x_test=test.iloc[:,np.r_[0:25]]
y_train=train.iloc[:,25]
y_test=test.iloc[:,25]
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


# Bidirectional LSTM

In [222]:
# covertir y_train de 2d a 3d
y_train3d = y_train.reshape((y_train.shape[0], n_steps, 1)) 

In [223]:
y_train3d.shape

(20631, 1, 1)

In [224]:
# covertir y_test de 2d a 3d
y_test3d = y_test.reshape((y_test.shape[0], n_steps, 1))
y_test3d.shape

(13096, 1, 1)

In [225]:
# diseñar la red bidireccional LSTM

from keras.layers import Bidirectional
from keras.layers import TimeDistributed

n_outputs=1
model_path_clf = 'regresion5_model.h5'  
model = Sequential()
model.add(Bidirectional(LSTM(200,return_sequences=True), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mae', optimizer='adam')

# entrenar la red
history = model.fit(x_train, y_train3d, epochs=100, batch_size=32, validation_data=(x_test, y_test3d), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())


Train on 20631 samples, validate on 13096 samples
Epoch 1/100
 - 11s - loss: 0.1094 - val_loss: 0.2020

Epoch 00001: val_loss improved from inf to 0.20200, saving model to regresion5_model.h5
Epoch 2/100
 - 7s - loss: 0.0944 - val_loss: 0.1731

Epoch 00002: val_loss improved from 0.20200 to 0.17307, saving model to regresion5_model.h5
Epoch 3/100
 - 7s - loss: 0.0912 - val_loss: 0.1225

Epoch 00003: val_loss improved from 0.17307 to 0.12247, saving model to regresion5_model.h5
Epoch 4/100
 - 7s - loss: 0.0886 - val_loss: 0.1111

Epoch 00004: val_loss improved from 0.12247 to 0.11105, saving model to regresion5_model.h5
Epoch 5/100
 - 7s - loss: 0.0874 - val_loss: 0.1062

Epoch 00005: val_loss improved from 0.11105 to 0.10617, saving model to regresion5_model.h5
Epoch 6/100
 - 7s - loss: 0.0866 - val_loss: 0.1036

Epoch 00006: val_loss improved from 0.10617 to 0.10358, saving model to regresion5_model.h5
Epoch 7/100
 - 7s - loss: 0.0861 - val_loss: 0.1009

Epoch 00007: val_loss improved

In [226]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion5_model.h5')

In [227]:
# prediccion
yhat = saved_model.predict(x_test)


In [228]:
yhat.shape

(13096, 1, 1)

In [229]:
yhat

array([[[0.49805832]],

       [[0.5158036 ]],

       [[0.50115514]],

       ...,

       [[0.04012182]],

       [[0.03145239]],

       [[0.01738974]]], dtype=float32)

In [230]:
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [231]:
y_test = y_test.reshape((len(y_test), 1))

In [232]:
y_test.shape

(13096, 1)

In [233]:
# convertir el pronostico a 2d
yhat = yhat.reshape((len(yhat), 1))

In [234]:
# calcular RMSE

rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.115


In [235]:
inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [236]:
inv_yhat.shape

(13096, 26)

In [237]:
inv_yhat[0]

array([0.49805832, 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [238]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [239]:
inv_yhat2

array([[ 1.80799053e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.87205091e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.81917005e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 1.54839783e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 1.23543119e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 7.27769777e+00,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [240]:
inv_yhat3 = inv_yhat2[:,0]

In [241]:
# y_test transformado
inv_yhat3

array([180.79905319, 187.20509076, 181.91700482, ...,  15.4839783 ,
        12.35431191,   7.27769777])

In [242]:

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [246]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 8.81
Mean Absolute Error (MAE): 30.64
------------------------------------------------------------
Mean Squared Error (MSE): 1714.39
Root Mean Squared Error (RMSE): 41.41
------------------------------------------------------------
Mean Percent Error (MPE): -1.23
Mean Absolute Percent Error (MAPE): 23.12
------------------------------------------------------------


El algoritmo con menor error relativo es la red stacked LSTM con un MAPE estimado del 22.72%, pero si tomamos en cuenta las dos metricas, el algoritmo de regresion con mejor desempeño es la red LSTM bidireccional (BLSTM) con un RMSE de 41.41 ciclos (unidad de medida de tiempo) y un MAPE de 23.12%.

### Preparar datos para el nuevo modelo

In [267]:
# Preparar datos de nuevo para el modelo, ya que se habian modificado por escalado y reshape
# no hay cambios en los datos, son las mismas lineas de la parte superior del codigo

columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
variables=['ciclo','set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21','ttf']
objetivo='ttf'
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])
x_train=train.iloc[:,np.r_[0:25]]
x_test=test.iloc[:,np.r_[0:25]]
y_train=train.iloc[:,25]
y_test=test.iloc[:,25]
x_test=x_test.to_numpy()
x_train=x_train.to_numpy()
y_test=y_test.to_numpy()
y_train=y_train.to_numpy()
n_steps = 1
x_train = x_train.reshape((x_train.shape[0], n_steps, x_train.shape[1])) 
x_test = x_test.reshape((x_test.shape[0], n_steps, x_test.shape[1]))


# Bidirectional LSTM ajustado

In [268]:
# covertir y_train de 2d a 3d
y_train3d = y_train.reshape((y_train.shape[0], n_steps, 1)) 

In [269]:
y_train3d.shape

(20631, 1, 1)

In [270]:
# covertir y_test de 2d a 3d
y_test3d = y_test.reshape((y_test.shape[0], n_steps, 1))
y_test3d.shape

(13096, 1, 1)

In [272]:
# diseñar la red bidireccional LSTM

n_outputs=1
model_path_clf = 'regresion7_model.h5'  
model = Sequential()
model.add(Bidirectional(LSTM(400,return_sequences=True), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(TimeDistributed(Dense(200, activation='relu')))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mae', optimizer='adam')

# entrenar la red
history = model.fit(x_train, y_train3d, epochs=100, batch_size=32, validation_data=(x_test, y_test3d), verbose=2, shuffle=False,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path_clf, monitor='val_loss', save_best_only=True, mode='min', verbose=2)])
print(history.history.keys())


Train on 20631 samples, validate on 13096 samples
Epoch 1/100
 - 14s - loss: 0.1124 - val_loss: 0.1772

Epoch 00001: val_loss improved from inf to 0.17725, saving model to regresion7_model.h5
Epoch 2/100
 - 8s - loss: 0.0971 - val_loss: 0.1942

Epoch 00002: val_loss did not improve from 0.17725
Epoch 3/100
 - 8s - loss: 0.0939 - val_loss: 0.1491

Epoch 00003: val_loss improved from 0.17725 to 0.14914, saving model to regresion7_model.h5
Epoch 4/100
 - 8s - loss: 0.0911 - val_loss: 0.1155

Epoch 00004: val_loss improved from 0.14914 to 0.11546, saving model to regresion7_model.h5
Epoch 5/100
 - 7s - loss: 0.0882 - val_loss: 0.1100

Epoch 00005: val_loss improved from 0.11546 to 0.11002, saving model to regresion7_model.h5
Epoch 6/100
 - 8s - loss: 0.0874 - val_loss: 0.1048

Epoch 00006: val_loss improved from 0.11002 to 0.10478, saving model to regresion7_model.h5
Epoch 7/100
 - 8s - loss: 0.0873 - val_loss: 0.1042

Epoch 00007: val_loss improved from 0.10478 to 0.10417, saving model to

In [273]:
# cargar el mejor modelo guardado durante el entrenamiento

saved_model = load_model('regresion7_model.h5')

In [274]:
# prediccion
yhat = saved_model.predict(x_test)


In [275]:
yhat.shape

(13096, 1, 1)

In [276]:
yhat

array([[[0.4982419 ]],

       [[0.5301308 ]],

       [[0.49586183]],

       ...,

       [[0.03285381]],

       [[0.03530513]],

       [[0.02005161]]], dtype=float32)

In [277]:
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
x_test.shape

(13096, 25)

In [278]:
y_test = y_test.reshape((len(y_test), 1))

In [279]:
y_test.shape

(13096, 1)

In [280]:
# convertir el pronostico a 2d
yhat = yhat.reshape((len(yhat), 1))

In [281]:
# calcular RMSE

rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.116


In [282]:
inv_yhat = concatenate((yhat, x_test[:, :]), axis=1)


In [283]:
inv_yhat.shape

(13096, 26)

In [284]:
inv_yhat[0]

array([0.4982419 , 0.        , 0.63218391, 0.75      , 0.        ,
       0.        , 0.54518072, 0.31066056, 0.26941256, 0.        ,
       1.        , 0.65217391, 0.21212121, 0.12761375, 0.        ,
       0.20833333, 0.64605544, 0.22058824, 0.13216018, 0.30896499,
       0.        , 0.33333333, 0.        , 0.        , 0.55813953,
       0.66183375])

In [285]:
inv_yhat2 = scaler.inverse_transform(inv_yhat)


In [286]:
inv_yhat2

array([[ 1.80865326e+02, -8.70000000e-03,  1.58620690e-04, ...,
         3.81400000e+01,  2.32984047e+01,  2.38921983e+02],
       [ 1.92377220e+02, -8.65180055e-03, -1.86206897e-04, ...,
         3.81400000e+01,  2.33882279e+01,  2.47944490e+02],
       [ 1.80006120e+02, -8.60360111e-03,  2.06896552e-05, ...,
         3.81400000e+01,  2.34219116e+01,  2.60406518e+02],
       ...,
       [ 1.28602261e+01,  6.98891967e-04, -4.13793103e-05, ...,
         3.81400000e+01,  2.31636698e+01,  1.54977769e+02],
       [ 1.37451510e+01,  7.47091413e-04, -2.62068966e-04, ...,
         3.81400000e+01,  2.31861256e+01,  1.87279343e+02],
       [ 8.23863246e+00,  7.95290859e-04,  8.96551724e-05, ...,
         3.81400000e+01,  2.32085814e+01,  1.45207539e+02]])

In [287]:
inv_yhat3 = inv_yhat2[:,0]

In [288]:
# y_test transformado
inv_yhat3

array([180.8653264 , 192.37722009, 180.00612003, ...,  12.86022612,
        13.74515103,   8.23863246])

In [289]:

inv_y = concatenate((y_test, x_test[:,:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
inv_y

array([143., 142., 141., ...,  23.,  22.,  21.])

In [290]:
regression_metrics(inv_y, inv_yhat3)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 10.71
Mean Absolute Error (MAE): 30.86
------------------------------------------------------------
Mean Squared Error (MSE): 1764.65
Root Mean Squared Error (RMSE): 42.01
------------------------------------------------------------
Mean Percent Error (MPE): 0.28
Mean Absolute Percent Error (MAPE): 23.11
------------------------------------------------------------


Teniendo en cuenta las tres métricas de desempeño se observa que los algoritmos con la mejor precisión (menor MAPE y RMSE) y con menor sesgo (MPE cercano a cero) son las redes neuronales profundas (DNN), específicamente las redes neuronales LSTM y el perceptrón multicapa; en el segundo grupo ordenados en términos de desempeño están los algoritmos de conjunto, por último, se presentan los algoritmos de ML lineales y no lineales con los mayores errores de los experimentos. 

El algoritmo con menor error relativo es la red apilada LSTM con un MAPE estimado del 22.72% y el algoritmo con menor error absoluto es Gradient Boosting con un RMSE estimado de 40.8 ciclos (unidad de medida de tiempo), pero si tomamos en cuenta las tres métricas, el algoritmo de regresión con mejor desempeño es la red LSTM bidireccional (BLSTM) con un MAPE de 23.11%, un RMSE de 42.01 ciclos y un MPE de 0.28% lo que indica un bajo sesgo de los valores estimados